# Maximum Likelihood
At the end of the previous section, we discussed the concept of estimating the parameters of the simple regression model. In this section, we will explore a method known as *maximum likelihood estimation* (MLE) that is applicable to both simple linear models, as well as many of the more complex models we will see later on the course.

## MLE vs OLS
At the beginning of this lesson, we demonstrated how the simple regression line could be fit by minimising the sum of squared errors. This is known as the method of *ordinary least squares* (OLS). OLS is useful because it is easy to conceptualise and results in a simple set of equations for finding estimates. Because of this, OLS is generally considered *the* method used for estimating linear models. However, models that do not assume a normal distribution or those that assume more complex correlational and variance structures cannot be estimated with OLS. As such, OLS is actually something of a niche approach that gets abandoned quickly as models get more complex. Because of this, it is much more helpful to consider a *likelihood* approach to estimation[^searlefoot]. The principle of the likelihood is widely applicable across may different types of model, as well as being fundamental to Bayesian methods. As such, we can consider MLE as our singular generic method of estimation that will be applicable across this entire course.

## How Does MLE Work?
Fundamentally, MLE is based on finding parameter estimates that make a value, known as the *likelihood*, as *big* as possible. The value of the likelihood is calculated using the *likelihood function*, denoted 

$$
L\left(\theta|y\right),
$$

where $\theta$ is a generic representation of the model parameters. For instance, in simple regression, $\theta = \{\beta_{0},\beta_{1},\sigma^{2}\}$. At its most basic, the likelihood function is a statement about the probability of the data we have collected. We can therefore write the likelihood in terms of probability like so

$$
L\left(\theta|y\right) = P\left(y|\theta\right).
$$

In words, this is saying that the likelihood of a set of parameter values, given some data, is the *same* as evaluating the probability of the data, assuming those parameter values are true. The key point here is that MLE is about the *probability of the data*[^bayesfoot]. We can think about it as taking a guess for the parameter values, then calculating how probable those guesses make the data we have collected. It is like asking the question: "how likely would it have been to collect my data, if these were the parameter values?" By searching through lots of different possible combinations of parameter values, the aim is to find the specific combination that leads to the *highest probability* of the data. These are then the values that *maxmimise the likelihood*.

### The Likelihood Function
So how do we evaluate the likelihood function? In brief, if we want the likelihood of our entire dataset, we need to *multiply* the probability of each datapoint, using some guesses for the parameters. We can then repeat this for some other guesses. If the likelihood gets *larger* then our new guesses make the data more probable than our old guesses. If we keep doing this until we cannot make the likelihood any bigger then we have found the "best" parameters, according to MLE.

So how do we calculate the probabilities of each data point? Remember that our core assumption for simple regression is that

$$
y_{i} \sim \mathcal{N}\left(\beta_{0} + \beta_{1}x_{i}, \sigma^{2}\right).
$$

The probability of any value from a normal distribution can be calculated using its *probability density function*, which is given by

$$
f(y) = \frac{1}{\sqrt{2\pi\sigma^{2}}}\exp\left(-\frac{(y - \mu)^{2}}{2\sigma^{2}}\right).
$$

This can look a little complicated, but notice that the only values we need to plug-in here are $\mu$, $\sigma^{2}$ and some value for $y$. For instance, if we wanted to know the probability of sampling a value of 10 from $\mathcal{N}(9,1)$, we can use

In [1]:
y      <- 10
mu     <- 9
sigma2 <- 1

P.y <- 1/sqrt(2*pi*sigma2) * exp(-((y-mu)^2)/(2*sigma2))
print(P.y)

[1] 0.2419707


By replacing $\mu$ with the mean function (and $\sigma^{2}$ with the variance function), we can reframe the probability density using our simple regression model to give

$$
P(y_{i}|x_{i},\theta) = \frac{1}{\sqrt{2\pi\sigma^{2}}}\exp\left(-\frac{\left(y_{i} - \left[\beta_{0} + \beta_{1}x_{i}\right]\right)^{2}}{2\sigma^{2}}\right).
$$

So this is the probability of the $i$th value of $y$, given *both* the $i$th value of the predictor variable *and* some guesses for the parameters. By plugging some values into $\theta = \left\{\beta_{0}, \beta_{1}, \sigma^{2}\right\}$, we can calculate the probability of any of our data values *given* our model of the data-generating process. This effectively tells us the probability of that data point, assuming our guesses are correct. 

Calculating the complete likelihood then involves multiplying all these values together to give

$$
L(\theta|\mathbf{y},\mathbf{x}) = P\left(y_{1}|x_{1},\theta\right) \times P(y_{2}|x_{2},\theta) \times \dots \times P(y_{n}|x_{n},\theta)
$$

which we can more compactly express using Big Pi notation (see the box below)

$$
\begin{align*}
    L(\theta|\mathbf{y},\mathbf{x}) &= \prod_{i=1}^{n} \frac{1}{\sqrt{2\pi\sigma^{2}}}\exp\left(-\frac{(y_{i} - [\beta_{0} + \beta_{1}x_{i}])^{2}}{2\sigma^{2}}\right) \\
    &= \prod_{i=1}^{n} P(y_{i}|x_{i},\theta)
\end{align*}
$$

`````{admonition} Big Pi Notation
:class: tip, dropdown
Big pi notation, denoted by the captitol Greek letter $\Pi$, is used as a shorthand for *multiplication*. Below the big Pi, we define our indexing variable, as well as its starting value. Above the big Pi, we indicate the value where we stop. So the notation

$$
P = \prod_{i=1}^{3} y_{i}
$$

is equivalent to

$$
P = y_{1} \times y_{2} \times y_{3} ,
$$

So, the notation says that our index is called $i$ and that we start it at $1$. We then keep going until $i = 3$, at which point we stop. 

This has a direct connection to a for-loop. So, in code, this is the same as shortening

```R
P <- y[1] * y[2] * y[3]
```

to

```R
P <- 1
for (i in 1:3){
    P <- P * y[i]
}
```
So, you can think of the Big Pi notation as a *multiplication loop* over a certain set of indices.
`````

### A Concrete Example in `R`
Given that expressing the likelihood in terms of probability density starts to look a bit complicated, we can turn to `R` to get a more concrete sense of these calculations. We will use the `mtcars` data again and, for the sake of argument, will set out guesses to $\beta_{0} = 30$, $\beta_{1} = -5$ and $\sigma^{2} = 1$. If we are assuming that the data have come from a normal distribution, we can calculate the probability of the first value of `mpg` using

In [2]:
beta.0 <- 30
beta.1 <- -5
sigma  <- 1

 y.1   <- mtcars$mpg[1]
mu.1   <- beta.0 + beta.1*mtcars$wt[1]
py.1   <- dnorm(y.1, mean=mu.1, sd=sigma)

print(py.1 )

[1] 8.926166e-05


where all the hideousness of the normal probability density is hidden behind the `dnorm()` function. The probability of the second value of `mpg` would then be

In [3]:
 y.2 <- mtcars$mpg[2]
mu.2 <- beta.0 + beta.1*mtcars$wt[2]
py.2 <- dnorm(y.2, mean=mu.2, sd=sigma)

print(py.2)

[1] 2.125155e-07


and so on. The find out the overall likelihood for the *whole* dataset, we just need to *multiply* these probabilities. However, this can cause computational problems, so we usually sum the *log* of these probabilities[^likprobsfoot]. This gives the *log-likelihood*.

In [4]:
mu     <- beta.0 + beta.1*mtcars$wt                            # means for whole dataset
loglik <- sum(dnorm(mtcars$mpg, mean=mu, sd=sigma2, log=TRUE)) # complete log likelihood

print(loglik)

[1] -780.7884


This value is not particularly interpretable, but this does not matter. All we want to do is make it as *big* as possible. In principle, we just need to keep trying guesses for the parameters to see which ones make the log-likelihood as large as possible. Once we cannot make it any bigger, the estimation is complete and we have found the values for the parameters that make the data as probable as possible.

### Exact Solutions
An obvious issue with the scheme above is that searching through many combinations of guesses for the parameters is not particularly efficient or practical. In fact, there is an infinite number of values we could choose, as well as an infinite number of combinations. So how can we possibly find the values we need? In order to do so, there are two options. For some simple problems, the equations that maximise the likelihood have already been derived using the tools of calculus. As such, the equation for the likelihood can be solved to find those values that guarantee a maximum. Normal linear models are one such example. By assuming a Gaussian distribution for the outcome variables, the ML estimates for a simple regression model are

$$
\begin{align*}
    \hat{\beta}_{1} &= \frac{\sum{\left(x_{i} - \bar{x}\right)\left(y_{i} - \bar{y}\right)}}{\sum{\left(x_{i} - \bar{x}\right)^{2}}}\\
    \hat{\beta}_{0} &= \bar{y} - \hat{\beta}_{1}\bar{x}, \\ 
\end{align*}
$$

which are identical to the solutions we saw earlier for OLS. As such, both MLE and OLS agree on the values of the intercept and slope.

### Iterative MLE
For situations where it is not possible to find an exact solution, we rely on computer algorithms to search through many possible combinations of values to find which one *maximises* the log-likelihood. These are known more generally as *optimisation* algorithms and are a large and complex topic in numerical computing. For our purpose, we do not really need to understand how these algorithms work. All we really need to know is that they use heuristics to explore the space of all possible parameter values in order to find those that the algorithm thinks make the log-likelihood the largest. We will see an example of this during the workshop next week.

### Restricted Maximum Likelihood
In the examples above, your may have noticed that we neglected to show the estimates for $\sigma^{2}$. This was not an accident. In terms of exact solutions, the variance estimates produced by OLS and MLE are

$$
\begin{align*}
    \hat{\sigma}^{2}_{\text{(OLS)}} &= \frac{1}{n - k} \sum_{i=1}^{n} e_{i}^{2} \\
    \hat{\sigma}^{2}_{\text{(ML)}}  &= \frac{1}{n} \sum_{i=1}^{n} e_{i}^{2} \\
\end{align*}
$$

So, the OLS variance estimate implements a generalisation of Bessel's correction. The value $n - k$ is known more generally as the *residual degrees of freedom*. If you are unfamiliar with concept of *degrees of freedom*, they are explored more in the drop-down box below. The ML estimate, by comparison, simply divides by $n$. Unless we have the entire population at our disposal, this will give a *biased* estimate for the variance.



`````{admonition} Degrees of Freedom
:class: tip, dropdown
The value $n-k$ in the denominator of the unbiased variance estimate is known as the *residual degrees of freedom*. If you have come across this concept before, you will known that it can be quite tricky to conceptualise. In effect, the degrees of freedom capture the number of independent pieces of information that are available for estimating some quantity. For example, consider a list of numbers of size $n$. If we then transform that list such that the sum of its values is 0, we are imposing a *constraint* on the number of possible values that list could contain. In this situation, $n-1$ of the items in the list are free to have any value we want. However, in order for the constraint to be met, the final item in the list will be *forced* to have a particular value. 

We can see this with an example. Take the following list of values

$$
y = \left[1,2,6,8\right].
$$

Now imagine that we transform $y$ to produce a new list, but impose the constraint that the sum of the values must be 0. One example of this would be

$$
y^{\ast} = \left[-3.25, -2.25, 1.75, 3.75\right].
$$

Notice here that the sum of the first 3 values is $-3.25 + -2.25 + 1.75 = -3.75$, which is the same as the final value with the sign flipped. This is no coincidence. In fact, we can choose any arbitrary set of values for the first 3 numbers. For instance

$$
y^{\ast} = \left[27, 4, -2, ?\right].
$$

The only limitation, is that the final value must negate the sum of these first 3 values, such that the overall sum is 0. This would give

$$
y^{\ast} = \left[27, 4, -2, -29\right].
$$

So in this example, $y$ has 4 degrees of freedom, but $y^{\ast}$ has $n-1 = 3$ degrees of freedom, because the value of element 4 is *constrained* by the fact it must sum to 0. This means that $y^{\ast}$ cannot consist of any numbers, rather it is constrained to be one of the set of numbers where the final value negates the sum of the first 3. The final value is therefore *dependent* on the first 3 and there are only 3 independent pieces of information here.

The importance of this for statistical models is that the act of estimation *imposes a constraint* on the values that are produced. Although our outcomes $y$ starts with $n$ degrees of freedom, estimating parameters limits the number of possibilities that can be produced by the model. For the residuals, their values are constrained by the parameter estimates. Each new parameter "uses up" a single degree of freedom. So if we have $k$ parameters, then the number of residual degrees of freedom is $n-k$. For simple regression, $k = 2$ (the slope and the intercept) and so the first $n-2$ residuals are free to be any possible value, but the value of the final 2 are constrained by the estimated values of $\beta_{0}$ and $\beta_{1}$. This also has a direct connection with Bessel's correction, because estimating the variance of a sample requires an estimate of the mean. This constrains the number of independent pieces of information that can be used to estimate the variance from $n$ to $n-1$.

The concept of degrees of freedom can be quite difficult to grasp. In fact, the origin of the concept came from Fisher's geometric perspective on linear models. Unfortunately, this requires a much deeper understanding of linear models that we will not have time to establish on this course. From this perpsective, the degrees of freedom actully correspond to the *number of dimensions* that numeric vectors exist within. Each of these vectors can only move within these dimensions, hence the connection with the mechanical concept of *motion* and *degrees of freedom*. This is not a perspective we would recommend trying to master at this stage. However, it is worth knowing that a more intuitive perspective on degrees of freedom does exist, if you want to look into it in the future.
`````

The reason this biased estimation happens is because MLE estimates the variance without taking into account that $\hat{\beta}_{0}$ and $\hat{\beta}_{1}$ are *estimates*. MLE treats these values as *known constants* and calculates the variance accordingly. As such, MLE is not *wrong* in its calculations, but it is wrong in its assumptions. In order to fix this, a variation of MLE was developed specifically for the unbiased estimation of variance, known as *restricted* maximum likelihood (REML). The way this works is a bit complicated and somewhat beyond the scope of this lesson. In effect, REML estimates the variance by first *removing* the effects of the predictors from the data. The resultant errors are then used to estimate the variance by running standard MLE. For this reason, this method is sometimes known as *residual* maximum likelihood. This removal allows the correction to be automatically taken into account, but how thus is achieved is quite complicated. Neverthless, this should give you enough of a flavour for REML and why it would be used in practice.

`````{topic} What do you now know?
In this section, we have explored the method of Maximum Likelihood as a means of estimating the simple regression parameters using a sample of data. After reading this section, you should have a good sense of:

- Why MLE is a more general-purpose and useful approach to focus on, even though OLS is typically used for estimating basic linear models.
- The concept that the likelihood function is really just a probability statement about the *data* given some estimates for the *parameters*.
- The concept that *maximising* the likelihood involves finding guesses for the parameters that makes the probability of the data as *large* as possible.
- The concept that there are *two* approaches to finding the parameters that maximise the likelihood. For well-behaved problems, exact solutions have already been found. For more difficult problems, iterative algorithms need to be used to search for solutions.
- The result that, for simple linear models assuming a normal distribution, the exact solutions for the slope and intercept are *identical* when using either MLE or OLS.
- The result that MLE produces a *biased* estimate of the variance and thus needs to be corrected using *restricted maximum likelihood* (REML). 
`````

[^likprobsfoot]: This is a problem of getting values of 0 due to issues with computational precision. Taking logs not only changes the scale so that this does not happen, but it also turns *multiplication* into *summation*. Historically, this made calculating the likelihood much easier by hand.

[^bayesfoot]: This may seem like the wrong quantity. Surely, we are interested in the parameter values with the highest probability, given the data we have? Unfortunately, evaluating $P(\boldsymbol{\theta}|\mathcal{D})$ requires Bayesian methods, which we will discuss in a litte more detail next week. Parameter estimation *without* Bayesian approaches therefore limits us to only evaluating $P(\mathcal{D}|\boldsymbol{\theta})$, even if this feels like the wrong thing to do.

[^searlefoot]: This is the perspective taken by [McCulloch, Searle & Neuhaus (2008)](https://www.librarysearch.manchester.ac.uk/permalink/44MAN_INST/1r887gn/alma9930787964401631), who are leading experts on linear models. The linked textbook is quite advanced, but does give you everything you would need to know about the theory of linear models. If you think these materials are hard, just have a little peek in there!